### Checking the functionality of `functions_list.py`

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("..")
import numpy as np
from qiskit import QuantumCircuit
import Dependencies.functions_list as df

In [29]:
n = 3
qc = QuantumCircuit(n)
qc.h([0,1,2])
qc.cz(0,1)
qc.s(1)
qc.h(1)
# qc.s(0)
qc.t(0)
qc.z(1)
qc.ccz(0,1,2)
d = qc.depth()
print(qc)

     ┌───┐   ┌───┐             
q_0: ┤ H ├─■─┤ T ├───────────■─
     ├───┤ │ ├───┤┌───┐┌───┐ │ 
q_1: ┤ H ├─■─┤ S ├┤ H ├┤ Z ├─■─
     ├───┤   └───┘└───┘└───┘ │ 
q_2: ┤ H ├───────────────────■─
     └───┘                     


In [41]:
instructions = [(instruction.operation.name,
                    [qc.find_bit(q).index for q in instruction.qubits]) 
                    for index, instruction in enumerate(qc.data)]
print(instructions)

[('h', [0]), ('h', [1]), ('h', [2]), ('cz', [0, 1]), ('s', [1]), ('h', [1]), ('t', [0]), ('z', [1]), ('ccz', [0, 1, 2])]


In [50]:
terms, wire_array, max_new_var = df.create_poly(qc, n)
print(terms)
print(wire_array)
print(max_new_var)
ovs = [sub[-1] for sub in wire_array]
print(ovs)

Initial wire_array:  [[0], [1], [2]]
[[4, [0, 3]], [4, [1, 4]], [4, [2, 5]], [4, [3, 4]], [2, [4]], [4, [4, 6]], [1, [3]], [4, [6]], [4, [3, 6, 5]]]
[[0, 3], [1, 4, 6], [2, 5]]
7
[3, 6, 5]


In [51]:
initial_state = np.zeros(n, dtype=bool)
print(initial_state)
tt = df.get_truthtable(terms, n, max_new_var, initial_state)
print(tt)

[False False False]
16
[0 4 0 4 2 2 2 2 1 5 1 1 7 7 7 3]


In [53]:
statevector = df.get_statevector(tt, n, max_new_var, ovs)
print(type(statevector))
print("Statevector using BOOLEAN equation: ")
threshold = 1e-8
for i, amp in enumerate(statevector):
    if abs(amp) > threshold:
        print(f"|{i:0{n}b}> : {amp:.6f}")

{0: array([1, 0, 1, 0, 0, 0, 0, 0]), 2: array([0, 0, 1, 0, 1, 0, 0, 0]), 1: array([1, 0, 1, 0, 0, 0, 0, 0]), 3: array([0, 0, 1, 0, 1, 0, 0, 0]), 4: array([0, 1, 0, 0, 0, 0, 0, 1]), 6: array([0, 0, 0, 0, 0, 1, 0, 1]), 5: array([0, 1, 0, 0, 0, 0, 0, 1]), 7: array([0, 1, 0, 1, 0, 0, 0, 0])}
<class 'numpy.ndarray'>
Statevector using BOOLEAN equation: 
|000> : 0.250000+0.250000j
|001> : 0.250000+0.250000j
|010> : -0.250000+0.250000j
|011> : -0.250000+0.250000j
|100> : 0.353553+0.000000j
|101> : 0.353553+0.000000j
|110> : 0.000000-0.353553j
|111> : 0.000000+0.353553j


In [45]:
from qiskit_aer import StatevectorSimulator
def get_stvec_aer(qc):
    backend = StatevectorSimulator()
    res = backend.run(qc).result()
    return res.get_statevector()

In [46]:
state_vector = np.asanyarray(get_stvec_aer(qc))
print("Statevector using aer simulator: ")
threshold = 1e-8
for i, amp in enumerate(state_vector):
    if abs(amp) > threshold:
        bits = format(i, f'0{qc.num_qubits}b')[::-1] # reversing the bits
        print(f"|{bits}> : {amp:.6f}")

Statevector using aer simulator: 
|000> : 0.250000+0.250000j
|100> : 0.353553+0.000000j
|010> : -0.250000+0.250000j
|110> : -0.000000-0.353553j
|001> : 0.250000+0.250000j
|101> : 0.353553+0.000000j
|011> : -0.250000+0.250000j
|111> : 0.000000+0.353553j


#### Using original Aarav's code of function's list file
It's working correctly

In [28]:
import functions_list_aarav as dfa
terms, wire_array, max_new_var = df.create_poly(qc, n)
ovs = [sub[-1] for sub in wire_array]
ivs = [i for i in range(n)]
tt = dfa.truthtable(terms, n, max_new_var, initial_state, ivs, np)
state_vector = dfa.statevector_(tt, n, max_new_var, ovs, np)
print(type(state_vector))
print("Statevector using boolean equation: ")
threshold = 1e-8
for i, amp in enumerate(state_vector):
    if abs(amp) > threshold:
        print(f"|{i:0{n}b}> : {amp:.6f}")

Initial wire_array:  [[0], [1], [2]]
<class 'numpy.ndarray'>
Statevector using boolean equation: 
|000> : 0.250000+0.250000j
|001> : 0.250000+0.250000j
|010> : -0.250000+0.250000j
|011> : -0.250000+0.250000j
|100> : 0.353553+0.000000j
|101> : 0.353553+0.000000j
|110> : 0.000000-0.353553j
|111> : 0.000000+0.353553j


#### Using DDSIM 

In [29]:
from mqt import ddsim
def get_stvec_ddsim(qc):
    backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
    job = backend.run(qc)
    result = job.result()
    return result.get_statevector()

In [32]:
state_vector = np.asanyarray(get_stvec_ddsim(qc))
print("Statevector using ddsim simulator: ")
threshold = 1e-8
for i, amp in enumerate(state_vector):
    if abs(amp) > threshold:
        bits = format(i, f'0{qc.num_qubits}b')[::-1] # reversing the bits
        print(f"|{bits}> : {amp:.6f}")

Statevector using ddsim simulator: 
|000> : 1.000000+0.000000j


### Checking random circuit generator

In [120]:
from Dependencies.random_circuit_generator import *

In [139]:
qc, qr = random_circ_d_const(5,5)
print(qc)

        ┌───┐        ┌───┐
q_0: ─■─┤ H ├─■───■──┤ H ├
      │ ├───┤ │   │  ├───┤
q_1: ─■─┤ H ├─■───■──┤ H ├
        ├───┤   ┌───┐├───┤
q_2: ─■─┤ H ├─■─┤ H ├┤ Z ├
      │ ├───┤ │ ├───┤├───┤
q_3: ─■─┤ Z ├─■─┤ H ├┤ H ├
      │ ├───┤ │ ├───┤├───┤
q_4: ─■─┤ Z ├─■─┤ H ├┤ Z ├
        └───┘   └───┘└───┘


In [140]:
qc, qr = random_circ_g_const(5,5)
print(qc)

          ┌───┐   
q_0: ─────┤ H ├───
          └───┘   
q_1: ───────■───■─
     ┌───┐  │   │ 
q_2: ┤ Z ├──┼───┼─
     └───┘  │   │ 
q_3: ───────■───■─
          ┌───┐   
q_4: ─────┤ H ├───
          └───┘   


In [228]:
qc, qr = random_circ_h_const(5,5,0.25)
print(qc)

                                                             
q_0: ───────■───────────────────────────■───────■──■─────────
     ┌───┐  │                           │ ┌───┐ │  │         
q_1: ┤ H ├──┼─────────────────────■──■──┼─┤ Z ├─■──┼──■──────
     └───┘  │                     │  │  │ └───┘ │  │  │ ┌───┐
q_2: ───────■────────■────────────■──■──┼───────┼──■──┼─┤ H ├
     ┌───┐┌───┐┌───┐ │ ┌───┐┌───┐ │     │       │  │  │ └───┘
q_3: ┤ H ├┤ Z ├┤ Z ├─■─┤ H ├┤ Z ├─■─────■───────┼──■──■──────
     ├───┤├───┤└───┘   └───┘└───┘       │       │            
q_4: ┤ Z ├┤ H ├─────────────────────────■───────■────────────
     └───┘└───┘                                              


### Checking random_circuit_gererator_universal

In [448]:
from Dependencies.random_circuit_generator_universal import *

In [449]:
qc, qr = random_circ_uni.h(3, 2)

print(qc)

             ┌─────┐                         
q_0: ──────■─┤ Tdg ├─────────────────────────
     ┌───┐ │ └┬───┬┘┌───┐   ┌─────┐┌───┐┌───┐
q_1: ┤ H ├─■──┤ Z ├─┤ Z ├─■─┤ Sdg ├┤ T ├┤ H ├
     └───┘    └───┘ └───┘ │ └─────┘└───┘└───┘
q_2: ─────────────────────■──────────────────
                                             
